In [5]:
import Pkg
Pkg.activate(raw"/Users/user/Desktop/Praca/Minnesota season 6/JOB MARKET/codes")
Pkg.instantiate()


  Activating project at `~/Desktop/Praca/Minnesota season 6/JOB MARKET/codes`


In [9]:
using LinearAlgebra
using NLsolve
using QuadGK
using Interpolations
using Statistics

# (Plot styling + warnings from Python are omitted; they don't affect results.)

# Constants
delta_0_star_initial = 0.47716112847660036
delta_1_star_initial = 0.11112649047308443
f  = 0.0048366897
s  = 0.6884162140
r  = 0.0361577940
gamma = 0.0867540457
rho   = 0.4133277998
lambda1 = 3.9750228339

delta_bar = 1.0
N_points = 101
delta_grid = collect(range(0.0, stop=delta_bar, length=N_points))
grid_spacing = delta_grid[2] - delta_grid[1]

# Override per later Python lines
f, gamma, rho, lambda1, delta_0_star_initial, delta_1_star_initial =
    0.021, 0.07, 0.3, 3.0, 0.483672706, 0.106481650304695

delta_bar = 1.0
N_points = 101
delta_grid = collect(range(0.0, stop=delta_bar, length=N_points))
grid_spacing = delta_grid[2] - delta_grid[1]

0.01

In [10]:


# --- Case functions for H1 ---
function H1_case_C(delta, F1_val, r, gamma, s, f, lambda1, rho)
    a = rho
    b = lambda1 * (f - F1_val) + gamma * (1 - delta) + rho * (1 - delta - s + F1_val) + gamma * delta
    c = -gamma * delta * (s - F1_val)
    discriminant = b^2 - 4a*c
    if a == 0
        return -c / b
    elseif discriminant >= 0
        return (-b + sqrt(discriminant)) / (2a)
    else
        return (-b) / (2a)
    end
end

function H1_case_D(delta, H1_delta1, F1_val, r, gamma, s, f, lambda1, rho)
    a = rho
    b = gamma * (1 - delta) + rho * (1 - delta - s + F1_val) + gamma * delta
    c = -gamma * delta * (s - F1_val) + lambda1 * (f - F1_val) * H1_delta1
    discriminant = b^2 - 4a*c
    if a == 0
        return -c / b
    elseif discriminant >= 0
        return (-b + sqrt(discriminant)) / (2a)
    else
        return (-b) / (2a)
    end
end

function H1_case_E(delta, H1_delta0, F1_val, r, gamma, s, f, lambda1, rho, H1_delta1, delta_0_star)
    a = rho
    b = gamma * (1 - delta) + rho * (1 - delta - s + F1_val) + lambda1 * F1_val + gamma * delta
    c = -(lambda1 * F1_val * (delta - delta_0_star + H1_delta0) + gamma * delta * (s - F1_val) - lambda1 * (f - F1_val) * H1_delta1)
    discriminant = b^2 - 4a*c
    if a == 0
        return -c / b
    elseif discriminant >= 0
        return (-b + sqrt(discriminant)) / (2a)
    else
        return (-b) / (2a)
    end
end

# --- Helper functions for H1 and H0 ---
function H1_delta1_comp(F1_val, r, gamma, s, f, lambda1, rho, delta_1_star)
    a = rho
    b = lambda1 * (f - F1_val) + gamma * (1 - delta_1_star) + rho * (1 - delta_1_star - s + F1_val) + gamma * delta_1_star
    c = -gamma * delta_1_star * (s - F1_val)
    if a == 0
        return -c / b
    else
        discriminant = b^2 - 4a*c
        if discriminant < 0
            error("Negative discriminant in quadratic equation.")
        end
        return (-b + sqrt(discriminant)) / (2a)
    end
end

function H1_delta0_comp(F1_val, H1_delta1, r, gamma, s, f, lambda1, rho, delta_0_star)
    a = rho
    b = gamma * (1 - delta_0_star) + rho * (1 - delta_0_star - s + F1_val) + gamma * delta_0_star
    c = -gamma * delta_0_star * (s - F1_val) + lambda1 * (f - F1_val) * H1_delta1
    if a == 0
        return -c / b
    else
        discriminant = b^2 - 4a*c
        if discriminant < 0
            error("Negative discriminant in quadratic equation.")
        end
        return (-b + sqrt(discriminant)) / (2a)
    end
end

function trade_condition(F1_val, r, gamma, s, f, lambda1, rho, delta_1_star, delta_0_star)
    H1_delta1 = H1_delta1_comp(F1_val, r, gamma, s, f, lambda1, rho, delta_1_star)
    H1_delta0 = H1_delta0_comp(F1_val, H1_delta1, r, gamma, s, f, lambda1, rho, delta_0_star)
    left_side  = F1_val * (1 - delta_0_star - s + F1_val + H1_delta0)
    right_side = (f - F1_val) * H1_delta1
    return left_side - right_side
end

# Solver for F1
function solve_F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    F1_initial = f
    function g!(F, x)
        F[1] = trade_condition(x[1], r, gamma, s, f, lambda1, rho, delta_1_star, delta_0_star)
    end
    sol = nlsolve(g!, [F1_initial]; xtol=1e-12, ftol=1e-12)
    return sol.zero[1]
end


F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho) =
    solve_F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)

F0(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho) =
    f - F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)

function H1(delta, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    H1_delta1 = H1_case_C(delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    H1_delta0 = H1_case_D(delta_0_star, H1_delta1, F1_val, r, gamma, s, f, lambda1, rho)
    if (0 < delta) && (delta <= delta_1_star)
        return H1_case_C(delta, F1_val, r, gamma, s, f, lambda1, rho)
    elseif (delta_1_star < delta) && (delta < delta_0_star)
        return H1_case_D(delta, H1_delta1, F1_val, r, gamma, s, f, lambda1, rho)
    elseif (delta_0_star <= delta) && (delta <= 1)
        return H1_case_E(delta, H1_delta0, F1_val, r, gamma, s, f, lambda1, rho, H1_delta1, delta_0_star)
    else
        return 0.0
    end
end

function H0(delta, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    if (0 < delta) && (delta <= 1)
        return delta - H1(delta, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    else
        return 0.0
    end
end

function compute_H_values(delta_grid, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    F1_val = F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    H1_values = Array{Float64}(undef, length(delta_grid))
    H0_values = Array{Float64}(undef, length(delta_grid))
    for (i, delta) in enumerate(delta_grid)
        H1_val = H1(delta, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
        H0_val = H0(delta, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
        if !(isfinite(H1_val))
            H1_val = 0.0
        end
        if !(isfinite(H0_val))
            H0_val = 0.0
        end
        H1_values[i] = H1_val
        H0_values[i] = H0_val
    end
    return H1_values, H0_values
end

function dH1(delta, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    F1_val = F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    F0_val = F0(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    H1_val = H1(delta, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)

    if (0 <= delta) && (delta <= delta_1_star)
        b = lambda1 * F0_val + gamma + rho * (1 - delta - s + F1_val + 2 * H1_val)
        c = gamma * (s - F1_val) + rho * H1_val
    elseif (delta_1_star < delta) && (delta < delta_0_star)
        b = gamma + rho * (1 - delta - s + F1_val + 2 * H1_val)
        c = gamma * (s - F1_val) + rho * H1_val
    elseif delta >= delta_0_star
        b = lambda1 * F1_val + gamma + rho * (1 - delta - s + F1_val + 2 * H1_val)
        c = lambda1 * F1_val + gamma * (s - F1_val) + rho * H1_val
    else
        return 0.0
    end
    return c / b
end

dH0(delta, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho) =
    1.0 - dH1(delta, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)

function dH1_gen(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    H1_values, _ = compute_H_values(delta_grid, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    dH1_values = zeros(length(delta_grid))
    F1_val = F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    F0_val = f - F1_val
    for (i, delta) in enumerate(delta_grid)
        H1_delta = H1_values[i]
        b = 0.0
        c = 0.0
        if (0 <= delta) && (delta <= delta_1_star)
            b = lambda1 * F0_val + gamma + rho * (1 - delta - s + F1_val + 2 * H1_delta)
            c = gamma * (s - F1_val) + rho * H1_delta
        elseif (delta_1_star < delta) && (delta < delta_0_star)
            b = gamma + rho * (1 - delta - s + F1_val + 2 * H1_delta)
            c = gamma * (s - F1_val) + rho * H1_delta
        elseif delta >= delta_0_star
            b = lambda1 * F1_val + gamma + rho * (1 - delta - s + F1_val + 2 * H1_delta)
            c = lambda1 * F1_val + gamma * (s - F1_val) + rho * H1_delta
        else
            dH1_values[i] = 0.0
            continue
        end
        dH1_values[i] = (b != 0.0) ? (c / b) : 0.0
    end
    return dH1_values
end

dH0_gen(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho) =
    1 .- dH1_gen(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)

function dH0Delta(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    dH0_values = dH0_gen(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    n = length(delta_grid)
    M = zeros(n, n)
    for i in 1:(n-1)
        M[i, (i+1):n] .= dH0_values[(i+1):n] .* grid_spacing
    end
    return M
end

function dH1Delta(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    dH1_values = dH1_gen(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    n = length(delta_grid)
    M = zeros(n, n)
    for i in 2:n
        M[i, 1:(i-1)] .= dH1_values[1:(i-1)] .* grid_spacing
    end
    return M
end

function Sigma(delta_grid, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    F1_val = F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    F0_val = f - F1_val
    sigma_values = zeros(length(delta_grid))
    H0_1 = H0(1.0, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    for (i, delta) in enumerate(delta_grid)
        H0_delta = H0(delta, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
        H1_delta = H1(delta, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
        if delta < delta_1_star
            sigma_values[i] = r + gamma + lambda1 * F0_val + (rho / 2) * (H0_1 - H0_delta) + (rho / 2) * H1_delta
        elseif (delta_1_star <= delta) && (delta <= delta_0_star)
            sigma_values[i] = r + gamma + (rho / 2) * (H0_1 - H0_delta) + (rho / 2) * H1_delta
        else # delta > delta_0_star
            sigma_values[i] = r + gamma + lambda1 * F1_val + (rho / 2) * (H0_1 - H0_delta) + (rho / 2) * H1_delta
        end
    end
    return diagm(0 => sigma_values)
end

function dG_outer_1T(delta_grid, r, gamma, s, f, lambda1, rho)
    n = length(delta_grid)
    dG = fill(1.0 / n, n)
    return dG * ones(n)'  # outer product
end

function W1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    F1_val = F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    H0_1 = H0(1.0, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    H0_delta0_star = H0(delta_0_star, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    H1_delta0_star = H1(delta_0_star, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    sigma_delta0 = r + gamma + (rho / 2) * (H0_1 - H0_delta0_star) + (rho / 2) * H1_delta0_star
    dH0_delta0 = dH0(delta_0_star, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    if dH0_delta0 == 0.0
        dH0_delta0 = 1e-7
    end
    return (lambda1 / r) * (H0_1 - H0_delta0_star)^2 / (sigma_delta0 * dH0_delta0)
end

function W0(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    F1_val = F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    H0_1 = H0(1.0, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    H0_delta1_star = H0(delta_1_star, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    H1_delta1_star = H1(delta_1_star, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
    sigma_delta1 = r + gamma + (rho / 2) * (H0_1 - H0_delta1_star) + (rho / 2) * H1_delta1_star
    dH1_delta1 = dH1(delta_1_star, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    if dH1_delta1 == 0.0
        dH1_delta1 = 1e-7
    end
    return (lambda1 / r) * H1_delta1_star^2 / (sigma_delta1 * dH1_delta1)
end

function value_iteration(delta_grid, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho;
                         max_iterations=3000, tolerance=1e-10)
    X_n = zeros(length(delta_grid))
    F1_val = F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    _H1_values, _H0_values = compute_H_values(delta_grid, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    _dH0_values = dH0_gen(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    _dH1_values = dH1_gen(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    dH0Delta_matrix = dH0Delta(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    dH1Delta_matrix = dH1Delta(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    Sigma_matrix = Sigma(delta_grid, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
    dG_outer_1T_matrix = dG_outer_1T(delta_grid, r, gamma, s, f, lambda1, rho)

    delta_1_index = findmin(abs.(delta_grid .- delta_1_star))[2]
    delta_0_index = findmin(abs.(delta_grid .- delta_0_star))[2]

    # (Not used in RHS below, but kept for parity with Python)
    n = length(delta_grid)
    matrix_1 = [ (j == delta_1_index && i < delta_1_index) ? 1.0 : 0.0 for i in 1:n, j in 1:n ]
    matrix_0 = [ (j == delta_0_index && i > delta_0_index) ? 1.0 : 0.0 for i in 1:n, j in 1:n ]

    for _iter in 1:max_iterations
        X_n_old = copy(X_n)
        Y = copy(delta_grid)
        indicator_delta_1 = [ d < delta_1_star ? 1.0 : 0.0 for d in delta_grid ]
        indicator_delta_0 = [ d > delta_0_star ? 1.0 : 0.0 for d in delta_grid ]

        RHS = Y .+ lambda1 * (f - F1_val) .* X_n[delta_1_index] .* indicator_delta_1 .+
                  lambda1 * F1_val .* X_n[delta_0_index] .* indicator_delta_0

        matrix_to_invert = Sigma_matrix .- gamma .* dG_outer_1T_matrix .- (rho / 2) .* dH0Delta_matrix .- (rho / 2) .* dH1Delta_matrix
        matrix_inv = inv(matrix_to_invert)
        X_n = matrix_inv * RHS

        if maximum(abs.(X_n .- X_n_old)) < tolerance
            return X_n, matrix_to_invert
        end
    end
    return X_n, Sigma_matrix .- gamma .* dG_outer_1T_matrix .- (rho / 2) .* dH0Delta_matrix .- (rho / 2) .* dH1Delta_matrix
end

function find_optimal_deltas_iterative(r, gamma, s, f, lambda1, rho, delta_grid, delta_0_star_initial,
                                       delta_1_star_initial; chi=0.03, tolerance=1e-3)
    max_iterations = 3000
    chi = min((rho != 0 ? r / lambda1 : r / lambda1), 1 / gamma)
    chi = 0.03
    start_time = time()

    delta_0_star = max(delta_0_star_initial, delta_1_star_initial)
    delta_1_star = min(delta_1_star_initial, delta_0_star_initial)

    for outer_iteration in 1:max_iterations
        Delta_V, matrix_to_invert = value_iteration(delta_grid, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)

        itp = interpolate((delta_grid,), Delta_V, Gridded(Linear()))
        Delta_V_interp = extrapolate(itp, Line())

        Sigma_matrix = Sigma(delta_grid, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)

        W_0 = W0(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)
        W_1 = W1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)

        F1_val = F1(delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho)

        H0_1 = H0(1.0, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
        H0_delta0_star = H0(delta_0_star, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
        H1_delta0_star = H1(delta_0_star, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
        H0_delta1_star = H0(delta_1_star, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)
        H1_delta1_star = H1(delta_1_star, delta_0_star, delta_1_star, F1_val, r, gamma, s, f, lambda1, rho)

        sigma_delta0 = r + gamma + (rho / 2) * (H0_1 - H0_delta0_star) + (rho / 2) * H1_delta0_star
        sigma_delta1 = r + gamma + (rho / 2) * (H0_1 - H0_delta1_star) + (rho / 2) * H1_delta1_star

        int1_delta0, _ = quadgk(δ -> Delta_V_interp(δ) * dH0(δ, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho), delta_0_star, 1.0; rtol=1e-10)
        int2_delta0, _ = quadgk(δ -> Delta_V_interp(δ) * dH1(δ, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho), 0.0, delta_0_star; rtol=1e-10)
        int3_delta0 = mean(Delta_V)

        int1_delta1, _ = quadgk(δ -> Delta_V_interp(δ) * dH0(δ, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho), delta_1_star, 1.0; rtol=1e-10)
        int2_delta1, _ = quadgk(δ -> Delta_V_interp(δ) * dH1(δ, delta_0_star, delta_1_star, r, gamma, s, f, lambda1, rho), 0.0, delta_1_star; rtol=1e-10)
        int3_delta1 = mean(Delta_V)

        if (H0_1 - H0_delta0_star) == 0.0
            H0_delta0_star = 1e-7 + H0_1
        end
        if H1_delta1_star == 0.0
            H1_delta1_star = 1e-7
        end

        delta_0_star_new = sigma_delta0 * ((1 + r / (lambda1 * (H0_1 - H0_delta0_star))) * W_1 - W_0) - (rho / 2) * int1_delta0 - (rho / 2) * int2_delta0 - gamma * int3_delta0
        delta_1_star_new = sigma_delta1 * (W_1 - (1 + r / (lambda1 * H1_delta1_star)) * W_0) - (rho / 2) * int1_delta1 - (rho / 2) * int2_delta1 - gamma * int3_delta1

        delta_0_star_new = min(max(delta_0_star_new, 0.0), 1.0)
        delta_1_star_new = min(max(delta_1_star_new, 0.0), 1.0)

        if (abs(delta_0_star_new - delta_0_star) < tolerance) && (abs(delta_1_star_new - delta_1_star) < tolerance)
            return delta_0_star, delta_1_star
        end

        delta_0_star = chi * delta_0_star_new + (1 - chi) * delta_0_star
        delta_1_star = chi * delta_1_star_new + (1 - chi) * delta_1_star
    end

    # (Keeps Python's prints but returns the final pair)
    println("Cutoffs Converged in $max_iterations iterations")
    println("Final delta_0_star: $delta_0_star, Final delta_1_star: $delta_1_star")
    return delta_0_star, delta_1_star
end

# --- P(δ, δ′), P0, P1 ---
P(delta, delta_prime, Delta_V_interp) = 0.5 * Delta_V_interp(delta) + 0.5 * Delta_V_interp(delta_prime)
P0(Delta_V_interp, delta_1_star) = Delta_V_interp(delta_1_star)
P1(Delta_V_interp, delta_0_star) = Delta_V_interp(delta_0_star)


P1 (generic function with 1 method)

In [14]:
start_time = time()
optimal_delta_0_star, optimal_delta_1_star =
    find_optimal_deltas_iterative(r, gamma, s, f, lambda1, rho, delta_grid,
                                  delta_0_star_initial, delta_1_star_initial;
                                  tolerance=1e-4)
println("Optimal delta_0_star: ", optimal_delta_0_star)
println("Optimal delta_1_star: ", optimal_delta_1_star)
elapsed_time = time() - start_time
println("Total time for operations: $(round(elapsed_time; digits=2)) seconds")


Optimal delta_0_star: 0.48399476856283485
Optimal delta_1_star: 0.10595321341448079
Total time for operations: 20.56 seconds
